## Setup

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# load model architecture
from tep.modelUtils import load_architecture
model = load_architecture('models/tl/baseline.json')
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seq_input (InputLayer)       (None, 32)                0         
_________________________________________________________________
word_emb (Embedding)         (None, 32, 100)           43224100  
_________________________________________________________________
input_dropout (Dropout)      (None, 32, 100)           0         
_________________________________________________________________
padding_1 (ZeroPadding1D)    (None, 34, 100)           0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 32, 128)           38528     
_________________________________________________________________
pool_1 (MaxPooling1D)        (None, 16, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 128)           0         
__________

In [3]:
# set model weights
model.load_weights('models/tl/baseline.hdf5')

In [4]:
# load data and labels
from tep.utils import load_array
seqs = load_array('data/sentiment_data/seq_32.bc')
labels = load_array('data/sentiment_data/labels.bc')
print(seqs.shape)
print(labels.shape)

(1600000, 32)
(1600000,)


In [5]:
# load word index
import json
word_index_file = open('data/sentiment_data/word_index.json')
word_index_str = word_index_file.read()
word_index = json.loads(word_index_str)
type(word_index)

dict

In [6]:
# compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
# test the model
from tep.trainUtils import one_hot_encoding
oh_labels = one_hot_encoding(labels, 2)
model.evaluate(x=seqs[:10000], y=oh_labels[:10000], batch_size=256)

10000/10000 [==============================] - 2s 218us/step


[0.34918261456489563, 0.84550000000000003]

## Visualize features

In [8]:
from tep.featureVisualization import ConvLayerVisualizer
clv = ConvLayerVisualizer(model, word_index)

In [9]:
clv.analyze_example(seqs[2528], 'conv_1', num_results=10)

[{'activation': 3.8374364,
  'kernel': 33,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 3.7581553,
  'kernel': 100,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 3.5078483,
  'kernel': 23,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 3.1374555,
  'kernel': 57,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 2.9925666,
  'kernel': 64,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 2.9566913,
  'kernel': 9,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 2.9120924,
  'kernel': 84,
  'position': 26,
  'words': 'all people smile'},
 {'activation': 2.8235338,
  'kernel': 65,
  'position': 27,
  'words': 'people smile in'},
 {'activation': 2.5399656,
  'kernel': 34,
  'position': 28,
  'words': 'smile in the'},
 {'activation': 2.4670651,
  'kernel': 127,
  'position': 28,
  'words': 'smile in the'}]

In [10]:
clv.analyze_neuron(seqs[:10000], 'conv_1', 100)

[{'activation': 3.7581553,
  'kernel': 100,
  'position': 27,
  'tweet': 2528,
  'words': 'people smile in'},
 {'activation': 3.5496619,
  'kernel': 100,
  'position': 17,
  'tweet': 473,
  'words': "i'm smiling !"},
 {'activation': 3.4549849,
  'kernel': 100,
  'position': 5,
  'tweet': 5774,
  'words': 'me smile .'},
 {'activation': 3.4549849,
  'kernel': 100,
  'position': 30,
  'tweet': 339,
  'words': 'me smile .'},
 {'activation': 3.4549849,
  'kernel': 100,
  'position': 30,
  'tweet': 7330,
  'words': 'me smile .'},
 {'activation': 3.4424214,
  'kernel': 100,
  'position': 21,
  'tweet': 3433,
  'words': '. thanks all'},
 {'activation': 3.4394569,
  'kernel': 100,
  'position': 27,
  'tweet': 9436,
  'words': '. glad it'},
 {'activation': 3.3911629,
  'kernel': 100,
  'position': 28,
  'tweet': 3419,
  'words': '. thankyouu <allcaps>'},
 {'activation': 3.3829739,
  'kernel': 100,
  'position': 31,
  'tweet': 6999,
  'words': 'me smile <unknown>'},
 {'activation': 3.3829739,
  '

## Calculate attributions

In [12]:
from deepexplain.tensorflow import DeepExplain
from keras import backend as K

In [18]:
model.layers[2].input

<tf.Tensor 'word_emb/Gather:0' shape=(?, 32, 100) dtype=float32>

In [ ]:
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    # Need to reconstruct the graph in DeepExplain context, using the same weights.
    # With Keras this is very easy:
    # 1. Get the input tensor to the original model
    input_tensor = model.layers[2].input
    
    # 2. We now target the output of the last dense layer (pre-softmax)
    # To do so, create a new model sharing the same layers untill the last dense (index -2)
    fModel = Model(inputs=input_tensor, outputs = model.output)
    target_tensor = fModel(input_tensor)
    
    xs = x_test[0:10]
    ys = y_test[0:10]
    
    attributions = de.explain('grad*input', target_tensor * ys, input_tensor, xs)
    #attributions = de.explain('saliency', target_tensor * ys, input_tensor, xs)
    #attributions = de.explain('intgrad', target_tensor * ys, input_tensor, xs)
    #attributions = de.explain('deeplift', target_tensor * ys, input_tensor, xs)
    #attributions = de.explain('elrp', target_tensor * ys, input_tensor, xs)
    #attributions = de.explain('occlusion', target_tensor * ys, input_tensor, xs)